<a href="https://colab.research.google.com/github/tdopsmang/Colab-projects/blob/main/Servicing_km_V1_0_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Did not work on RR path, rather added bus details upto DP, in consolidated report, also batch processing done in few cases

In [2]:
!pip install --upgrade gspread
!pip install gspread google-auth-oauthlib google-auth-httplib2
!pip install gspread google-auth oauth2client
!pip install gspread oauth2client pandas matplotlib seaborn

In [3]:
from google.colab import auth
auth.authenticate_user()
import pandas as pd
import gspread
from google.auth import default
creds, _ = default()
from google.oauth2.service_account import Credentials
gc = gspread.authorize(creds)
from oauth2client.client import GoogleCredentials
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
from collections import defaultdict
import re
import random
import time
from collections import defaultdict
import concurrent.futures
from functools import lru_cache

In [4]:
# Details our Database spreadsheet which has details of all files, ID in Transport Department

Database_File_spreadsheet_ID = gc.open_by_key('1nJKyvV1WQmZzvbjOP7Hsp1bQJmiNsYoJOH_jIL7H9PI') #ID of Database Spreadsheet
MASsheetID = Database_File_spreadsheet_ID.worksheet('MAS')                                    #MAS Worksheet
OdometersheetID = Database_File_spreadsheet_ID.worksheet('Odometer')                          #Odometer Worksheet
ReportID = Database_File_spreadsheet_ID.worksheet('Report')                          #Odometer Worksheet

#Reports spreadsheet and worksheet details
Report_EO_SpreadsheetID = ReportID.acell('C2').value                  # Get the spreadsheet ID from cell C2
Report_EO = gc.open_by_key(Report_EO_SpreadsheetID)                    # Open RR_Oil_Lub spreadsheet
Report_EO1_worksheet_name = ReportID.acell('B2').value                        # Km after last servicing consolidated
Report_EO1_Worksheet = Report_EO.worksheet(Report_EO1_worksheet_name)  # Open RR CH engineoil worksheet

#STS RR SVP
RR_Oil_Lub_Coolant_DEF_spreadsheet_id = MASsheetID.acell('C2').value                  # Get the spreadsheet ID from cell C2
RR_Oil_Lub = gc.open_by_key(RR_Oil_Lub_Coolant_DEF_spreadsheet_id)                    # Open RR_Oil_Lub spreadsheet

RR_Engine_oil_CH_BSIV_worksheet_name = MASsheetID.acell('B2').value                        # Get the worksheet name from cell B2
engine_oil_CH_BSIV_CH_MAS_rr = RR_Oil_Lub.worksheet(RR_Engine_oil_CH_BSIV_worksheet_name)  # Open RR CH engineoil worksheet

RR_Engine_oil_CK_BSVI_worksheet_name = MASsheetID.acell('B3').value                        # Get the worksheet name from cell B3
engine_oil_CK_BSVI_CK_MAS_rr = RR_Oil_Lub.worksheet(RR_Engine_oil_CK_BSVI_worksheet_name)  # Open RR CK engineoil worksheet

RR_Report1_worksheet_name = MASsheetID.acell('B8').value                                # Get the worksheet name from report1, RR Km between and after last servicing
RR_Report1_Worksheet = RR_Oil_Lub.worksheet(RR_Report1_worksheet_name)                  # Open RR Km between and after last servicing worksheet

RR_Report1_worksheet_name = MASsheetID.acell('B8').value                                # Get the worksheet name from report1, RR Km between and after last servicing
RR_Report1_Worksheet = RR_Oil_Lub.worksheet(RR_Report1_worksheet_name)                  # Open RR Km between and after last servicing worksheet

#STS ATR SVP
ATR_Oil_Lub_Coolant_DEF_spreadsheet_id = MASsheetID.acell('C52').value                  # Get the spreadsheet ID from cell C51 ATR
ATR_Oil_Lub = gc.open_by_key(ATR_Oil_Lub_Coolant_DEF_spreadsheet_id)                    # Open ATR_Oil_Lub spreadsheet

ATR_Engine_oil_CH_BSIV_worksheet_name = MASsheetID.acell('B52').value                         # Get the worksheet name from cell B52
engine_oil_CH_BSIV_CH_MAS_ATR = ATR_Oil_Lub.worksheet(ATR_Engine_oil_CH_BSIV_worksheet_name)  # Open ATR CH engineoil worksheet

ATR_Engine_oil_CK_BSVI_worksheet_name = MASsheetID.acell('B53').value                         # Get the worksheet name from cell B52
engine_oil_CK_BSVI_CK_MAS_ATR = ATR_Oil_Lub.worksheet(ATR_Engine_oil_CK_BSVI_worksheet_name)  # Open ATR CK engineoil worksheet

#ATR_Report1_worksheet_name = MASsheetID.acell('B58').value                           #******This may not be required       # Get the worksheet name from report1, RR Km between and after last servicing
#ATR_Report1_Worksheet = ATR_Oil_Lub.worksheet(ATR_Report1_worksheet_name)              #This may not be required     # Open RR Km between and after last servicing worksheet

#STS FG
FG_Oil_Lub_Coolant_DEF_spreadsheet_id = MASsheetID.acell('C102').value                  # Get the spreadsheet ID from cell C51 ATR
FG_Oil_Lub = gc.open_by_key(FG_Oil_Lub_Coolant_DEF_spreadsheet_id)                    # Open ATR_Oil_Lub spreadsheet

FG_Engine_oil_CH_BSIV_worksheet_name = MASsheetID.acell('B102').value                         # Get the worksheet name from cell B2
engine_oil_CH_BSIV_CH_MAS_FG = FG_Oil_Lub.worksheet(FG_Engine_oil_CH_BSIV_worksheet_name)  # Open RR engineoil worksheet

FG_Engine_oil_CK_BSVI_worksheet_name = MASsheetID.acell('B103').value                         # Get the worksheet name from cell B52
engine_oil_CK_BSVI_CK_MAS_FG = FG_Oil_Lub.worksheet(FG_Engine_oil_CK_BSVI_worksheet_name)  # Open ATR CK engineoil worksheet

#FG_Report1_worksheet_name = MASsheetID.acell('B108').value                    #*****This may not be required            # Get the worksheet name from report1, RR Km between and after last servicing
#FG_Report1_Worksheet = FG_Oil_Lub.worksheet(FG_Report1_worksheet_name)          #This may not be required         # Open RR Km between and after last servicing worksheet


# Process SVP data
SVP_Odometer_id = OdometersheetID.acell('A2').value
Odometer_spreadsheet_SVP = gc.open_by_key(SVP_Odometer_id)
bus_number_SVP = sorted([worksheet.title for worksheet in Odometer_spreadsheet_SVP])
spreadsheet_name_SVP = Odometer_spreadsheet_SVP.title
start_row_svp = 2

# Process FG data
FG_Odometer_id = OdometersheetID.acell('B2').value
Odometer_spreadsheet_FG = gc.open_by_key(FG_Odometer_id)
bus_number_FG = sorted([worksheet.title for worksheet in Odometer_spreadsheet_FG])
spreadsheet_name_FG = Odometer_spreadsheet_FG.title
start_row_fg = start_row_svp + len(bus_number_SVP)

# Process Baratang data
BT_Odometer_id = OdometersheetID.acell('C2').value
Odometer_spreadsheet_BT = gc.open_by_key(BT_Odometer_id)
bus_number_BT = sorted([worksheet.title for worksheet in Odometer_spreadsheet_BT])
spreadsheet_name_BT = Odometer_spreadsheet_BT.title
start_row_bt = start_row_fg + len(bus_number_FG)

# Process Rangat data
RT_Odometer_id = OdometersheetID.acell('D2').value
Odometer_spreadsheet_RT = gc.open_by_key(RT_Odometer_id)
bus_number_RT = sorted([worksheet.title for worksheet in Odometer_spreadsheet_RT])
spreadsheet_name_RT = Odometer_spreadsheet_RT.title
start_row_rt = start_row_bt + len(bus_number_BT)

# Process Mayabunder data
MB_Odometer_id = OdometersheetID.acell('E2').value
Odometer_spreadsheet_MB = gc.open_by_key(MB_Odometer_id)
bus_number_MB = sorted([worksheet.title for worksheet in Odometer_spreadsheet_MB])
spreadsheet_name_MB = Odometer_spreadsheet_MB.title
start_row_mb = start_row_rt + len(bus_number_RT)

# Process Diglipur data
DP_Odometer_id = OdometersheetID.acell('F2').value
Odometer_spreadsheet_DP = gc.open_by_key(DP_Odometer_id)
bus_number_DP = sorted([worksheet.title for worksheet in Odometer_spreadsheet_DP])
spreadsheet_name_DP = Odometer_spreadsheet_DP.title
start_row_dp = start_row_mb + len(bus_number_MB)



Below codes are redundent, No need to run, Use this If required in future

In [ ]:
# High speed, RR, code of 1.0.7 is fast but only for CH, if we incorporate CK, it will be super fast

import re
from datetime import datetime
import random
import time
from collections import defaultdict
import gspread
import math
from concurrent.futures import ThreadPoolExecutor

# List of all worksheets to process
engine_oil_worksheets = [
    'engine_oil_CH_BSIV_CH_MAS_rr',
    'engine_oil_CK_BSVI_CK_MAS_rr'
]

# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

# Function to process each worksheet
def process_worksheet(worksheet_name):
    print(f"\n🔄 Processing worksheet: {worksheet_name}")

    # Get the worksheet object
    worksheet = None
    try:
        # Assuming these are defined in your notebook environment
        worksheet = eval(worksheet_name)  # This will use the already defined variable in your notebook
    except:
        print(f"❌ Error: Worksheet '{worksheet_name}' not found or not accessible")
        return

    # Clear data in range L2:M, Odometer value in Engine Oil and Fallback date removal
    clear_range = 'L2:M'
    print(f"🧹 Clearing range {clear_range} in {worksheet_name}...")
    worksheet.batch_clear([clear_range])

    print("🔍 Starting data processing with date fallback search...")

    # 📥 Load main sheet
    print("📥 Loading worksheet data...")
    main_data = worksheet.get_all_values()
    print(f"✅ Loaded {len(main_data)-1} rows from worksheet")

    # Step 1: Collect unique (bus, date) and row mapping
    print("🔢 Mapping bus numbers and dates...")
    bus_date_map = defaultdict(set)
    row_map = {}  # maps row number → (bus, date)

    for i, row in enumerate(main_data[1:], start=2):
        try:
            date_str = row[1].strip()
            bus_num = row[4].strip()
            if date_str and bus_num:
                bus_date_map[bus_num].add(date_str)
                row_map[i] = (bus_num, date_str)
        except:
            continue

    print(f"✅ Found {len(bus_date_map)} unique bus numbers")
    print(f"✅ Found {len(row_map)} rows to process")

    # Create a cache for worksheet data to avoid redundant fetches
    bus_worksheet_cache = {}

    # Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
    print("\n🔍 Searching for exact dates or closest earlier dates...")
    bus_date_value_map = {}
    match_type_map = {}  # Tracks whether each match is exact or fallback
    fallback_date_map = {}  # Store the actual fallback date used
    success_count = 0
    fallback_count = 0

    # Function to process a single bus
    def process_bus(bus_dates_tuple):
        bus, dates = bus_dates_tuple
        sheet_success = 0
        results = []

        try:
            # Check if we already have this bus's data in cache
            if bus in bus_worksheet_cache:
                sheet_data = bus_worksheet_cache[bus]
            else:
                # Fetch the bus worksheet data and store in cache
                sheet = Odometer_spreadsheet_SVP.worksheet(bus)
                sheet_data = sheet.get_all_values()
                bus_worksheet_cache[bus] = sheet_data

            # Create a dictionary for faster exact date lookups
            date_value_lookup = {}
            for row in sheet_data[1:]:
                try:
                    if len(row) > 12:
                        date_str_key = row[1].strip()
                        date_value_lookup[date_str_key] = row[12]  # Column M
                except:
                    continue

            for date_str in dates:
                # Check for exact match using the lookup dictionary
                exact_value = date_value_lookup.get(date_str)

                if exact_value:
                    results.append((bus, date_str, exact_value, "exact", ""))
                    sheet_success += 1
                else:
                    # Try to find closest earlier date
                    value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                    if value:
                        results.append((bus, date_str, value, "fallback", fallback_date))
                        sheet_success += 1

            print(f"  ✅ Found {sheet_success}/{len(dates)} values for bus {bus}")
            return results

        except Exception as e:
            print(f"❌ Could not process sheet '{bus}': {str(e)}")
            return []

    # Process buses in parallel using ThreadPoolExecutor
    print(f"🚀 Processing {len(bus_date_map)} buses in parallel...")

    # Convert to list for ThreadPoolExecutor
    bus_dates_list = list(bus_date_map.items())

    # Use ThreadPoolExecutor for parallel processing
    # Adjust max_workers based on your system's capabilities
    with ThreadPoolExecutor(max_workers=5) as executor:
        all_results = list(executor.map(process_bus, bus_dates_list))

    # Process all results
    for bus_results in all_results:
        for bus, date_str, value, match_type, fallback_date in bus_results:
            bus_date_value_map[(bus, date_str)] = value
            match_type_map[(bus, date_str)] = match_type
            fallback_date_map[(bus, date_str)] = fallback_date

            if match_type == "exact":
                success_count += 1
            else:
                fallback_count += 1

    print(f"\n✅ Found values for {success_count} exact date matches")
    print(f"✅ Found values for {fallback_count} fallback date matches")
    print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

    # Step 3: Update values into Column L of main sheet and fallback dates into Column M
    print(f"\n📝 Preparing batch updates for {worksheet_name}...")

    # Prepare batch update for all cells at once
    batch_updates = []
    exact_match_cells = []
    fallback_match_cells = []

    for row_num, (bus, date_str) in row_map.items():
        value = bus_date_value_map.get((bus, date_str), "")
        fallback_date = fallback_date_map.get((bus, date_str), "")

        if value:
            # Add to batch updates list (row, col, value)
            batch_updates.append({
                'row': row_num,
                'col': 12,  # Column L
                'value': value
            })

            # Add fallback date update
            batch_updates.append({
                'row': row_num,
                'col': 13,  # Column M
                'value': fallback_date
            })

            # Track which cells need which formatting
            match_type = match_type_map.get((bus, date_str))
            if match_type == "exact":
                exact_match_cells.append(f"L{row_num}")
            elif match_type == "fallback":
                fallback_match_cells.append(f"L{row_num}")

    print(f"✅ Prepared {len(batch_updates)} cell updates")

    # Execute batch updates in chunks to avoid rate limits
    updated_count = 0
    chunk_size = 50  # Adjust based on API limits
    chunks = math.ceil(len(batch_updates) / chunk_size)

    print(f"🔄 Executing batch updates in {chunks} chunks...")

    for chunk_index in range(chunks):
        start_idx = chunk_index * chunk_size
        end_idx = min(start_idx + chunk_size, len(batch_updates))
        current_chunk = batch_updates[start_idx:end_idx]

        max_retries = 5
        retry_count = 0
        update_successful = False

        while not update_successful and retry_count < max_retries:
            try:
                # Prepare the batch update request
                cell_list = []

                for update in current_chunk:
                    cell = worksheet.cell(update['row'], update['col'])
                    cell.value = update['value']
                    cell_list.append(cell)

                # Execute the batch update
                worksheet.update_cells(cell_list, value_input_option='USER_ENTERED')

                update_successful = True
                updated_count += len(current_chunk) // 2  # Divide by 2 because we have 2 cells per row
                print(f"📊 Progress: Chunk {chunk_index+1}/{chunks} complete - {updated_count}/{len(batch_updates)//2} rows updated")

                # Add a small delay between chunks to avoid rate limits
                time.sleep(1)

            except Exception as e:
                retry_count += 1
                if "429" in str(e):
                    wait_time = (2 ** retry_count) + random.random()
                    print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Failed to update chunk {chunk_index+1}: {e}")
                    break

        if not update_successful:
            print(f"❌ Failed to update chunk {chunk_index+1} after {max_retries} retries")

    # Apply formatting in batches (currently commented out)
    print("\n🎨 Applying formatting to distinguish match types...")
    # You can uncomment and implement batch formatting logic if needed.

    print(f"\n✅ COMPLETE for {worksheet_name}: Updated {updated_count} rows in column L and column M")
    print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
    print(f"❌ {len(row_map) - updated_count} rows could not be updated")

# Main execution
print("🚀 Starting to process all engine oil worksheets...")

# Create a global cache for bus worksheets to reuse across all worksheets
global_bus_worksheet_cache = {}

# Process each worksheet in sequence
for worksheet_name in engine_oil_worksheets:
    print(f"\n{'='*80}\n📊 Processing worksheet: {worksheet_name}\n{'='*80}")
    process_worksheet(worksheet_name)
    # Add a delay between worksheets to avoid rate limiting
    time.sleep(2)

print("\n✅ All worksheets have been processed successfully!")
print("💡 Performance Summary:")
print(f"✓ Processed {len(engine_oil_worksheets)} worksheets with batch processing")
print(f"✓ Used parallel processing for bus data with ThreadPoolExecutor")
print(f"✓ Implemented caching to reduce API calls")
print(f"✓ Used batch updates to minimize API requests")


In [ ]:
# ATR , (RR, code of 1.0.7 is fast but only for CH, if we incorporate CK, and adopt for ATR and FG it will be super fast)


# Modified to process multiple engine oil worksheets, Added batch processing done

# Modified to process multiple engine oil worksheets with batch processing
# With fallback date display in M Column, For ATR_MAS_Engine Oil

# Make sure to include these imports at the top of your code
import re
from datetime import datetime
import random
import time
from collections import defaultdict
import gspread
import math
from concurrent.futures import ThreadPoolExecutor

# List of all worksheets to process
engine_oil_worksheets = [
    'engine_oil_CH_BSIV_CH_MAS_ATR',
    'engine_oil_CK_BSVI_CK_MAS_ATR'
]

# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

# Function to process each worksheet
def process_worksheet(worksheet_name):
    print(f"\n🔄 Processing worksheet: {worksheet_name}")

    # Get the worksheet object
    worksheet = None
    try:
        # Assuming these are defined in your notebook environment
        worksheet = eval(worksheet_name)  # This will use the already defined variable in your notebook
    except:
        print(f"❌ Error: Worksheet '{worksheet_name}' not found or not accessible")
        return

    # Clear data in range L2:M, Odometer value in Engine Oil and Fallback date removal
    clear_range = 'L2:M'
    print(f"🧹 Clearing range {clear_range} in {worksheet_name}...")
    worksheet.batch_clear([clear_range])

    print("🔍 Starting data processing with date fallback search...")

    # 📥 Load main sheet
    print("📥 Loading worksheet data...")
    main_data = worksheet.get_all_values()
    print(f"✅ Loaded {len(main_data)-1} rows from worksheet")

    # Step 1: Collect unique (bus, date) and row mapping
    print("🔢 Mapping bus numbers and dates...")
    bus_date_map = defaultdict(set)
    row_map = {}  # maps row number → (bus, date)

    for i, row in enumerate(main_data[1:], start=2):
        try:
            date_str = row[1].strip()
            bus_num = row[4].strip()
            if date_str and bus_num:
                bus_date_map[bus_num].add(date_str)
                row_map[i] = (bus_num, date_str)
        except:
            continue

    print(f"✅ Found {len(bus_date_map)} unique bus numbers")
    print(f"✅ Found {len(row_map)} rows to process")

    # Create a cache for worksheet data to avoid redundant fetches
    bus_worksheet_cache = {}

    # Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
    print("\n🔍 Searching for exact dates or closest earlier dates...")
    bus_date_value_map = {}
    match_type_map = {}  # Tracks whether each match is exact or fallback
    fallback_date_map = {}  # Store the actual fallback date used
    success_count = 0
    fallback_count = 0

    # Function to process a single bus
    def process_bus(bus_dates_tuple):
        bus, dates = bus_dates_tuple
        sheet_success = 0
        results = []

        try:
            # Check if we already have this bus's data in cache
            if bus in bus_worksheet_cache:
                sheet_data = bus_worksheet_cache[bus]
            else:
                # Fetch the bus worksheet data and store in cache
                sheet = Odometer_spreadsheet_SVP.worksheet(bus)
                sheet_data = sheet.get_all_values()
                bus_worksheet_cache[bus] = sheet_data

            # Create a dictionary for faster exact date lookups
            date_value_lookup = {}
            for row in sheet_data[1:]:
                try:
                    if len(row) > 12:
                        date_str_key = row[1].strip()
                        date_value_lookup[date_str_key] = row[12]  # Column M
                except:
                    continue

            for date_str in dates:
                # Check for exact match using the lookup dictionary
                exact_value = date_value_lookup.get(date_str)

                if exact_value:
                    results.append((bus, date_str, exact_value, "exact", ""))
                    sheet_success += 1
                else:
                    # Try to find closest earlier date
                    value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                    if value:
                        results.append((bus, date_str, value, "fallback", fallback_date))
                        sheet_success += 1

            print(f"  ✅ Found {sheet_success}/{len(dates)} values for bus {bus}")
            return results

        except Exception as e:
            print(f"❌ Could not process sheet '{bus}': {str(e)}")
            return []

    # Process buses in parallel using ThreadPoolExecutor
    print(f"🚀 Processing {len(bus_date_map)} buses in parallel...")

    # Convert to list for ThreadPoolExecutor
    bus_dates_list = list(bus_date_map.items())

    # Use ThreadPoolExecutor for parallel processing
    # Adjust max_workers based on your system's capabilities
    with ThreadPoolExecutor(max_workers=5) as executor:
        all_results = list(executor.map(process_bus, bus_dates_list))

    # Process all results
    for bus_results in all_results:
        for bus, date_str, value, match_type, fallback_date in bus_results:
            bus_date_value_map[(bus, date_str)] = value
            match_type_map[(bus, date_str)] = match_type
            fallback_date_map[(bus, date_str)] = fallback_date

            if match_type == "exact":
                success_count += 1
            else:
                fallback_count += 1

    print(f"\n✅ Found values for {success_count} exact date matches")
    print(f"✅ Found values for {fallback_count} fallback date matches")
    print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

    # Step 3: Update values into Column L of main sheet and fallback dates into Column M
    print(f"\n📝 Preparing batch updates for {worksheet_name}...")

    # Prepare batch update for all cells at once
    batch_updates = []
    exact_match_cells = []
    fallback_match_cells = []

    for row_num, (bus, date_str) in row_map.items():
        value = bus_date_value_map.get((bus, date_str), "")
        fallback_date = fallback_date_map.get((bus, date_str), "")

        if value:
            # Add to batch updates list (row, col, value)
            batch_updates.append({
                'row': row_num,
                'col': 12,  # Column L
                'value': value
            })

            # Add fallback date update
            batch_updates.append({
                'row': row_num,
                'col': 13,  # Column M
                'value': fallback_date
            })

            # Track which cells need which formatting
            match_type = match_type_map.get((bus, date_str))
            if match_type == "exact":
                exact_match_cells.append(f"L{row_num}")
            elif match_type == "fallback":
                fallback_match_cells.append(f"L{row_num}")

    print(f"✅ Prepared {len(batch_updates)} cell updates")

    # Execute batch updates in chunks to avoid rate limits
    updated_count = 0
    chunk_size = 50  # Adjust based on API limits
    chunks = math.ceil(len(batch_updates) / chunk_size)

    print(f"🔄 Executing batch updates in {chunks} chunks...")

    for chunk_index in range(chunks):
        start_idx = chunk_index * chunk_size
        end_idx = min(start_idx + chunk_size, len(batch_updates))
        current_chunk = batch_updates[start_idx:end_idx]

        max_retries = 5
        retry_count = 0
        update_successful = False

        while not update_successful and retry_count < max_retries:
            try:
                # Prepare the batch update request
                cell_list = []

                for update in current_chunk:
                    cell = worksheet.cell(update['row'], update['col'])
                    cell.value = update['value']
                    cell_list.append(cell)

                # Execute the batch update
                worksheet.update_cells(cell_list, value_input_option='USER_ENTERED')

                update_successful = True
                updated_count += len(current_chunk) // 2  # Divide by 2 because we have 2 cells per row
                print(f"📊 Progress: Chunk {chunk_index+1}/{chunks} complete - {updated_count}/{len(batch_updates)//2} rows updated")

                # Add a small delay between chunks to avoid rate limits
                time.sleep(1)

            except Exception as e:
                retry_count += 1
                if "429" in str(e):
                    wait_time = (2 ** retry_count) + random.random()
                    print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Failed to update chunk {chunk_index+1}: {e}")
                    break

        if not update_successful:
            print(f"❌ Failed to update chunk {chunk_index+1} after {max_retries} retries")

    # Apply formatting in batches (currently commented out)
    print("\n🎨 Applying formatting to distinguish match types...")
    # You can uncomment and implement batch formatting logic if needed.

    print(f"\n✅ COMPLETE for {worksheet_name}: Updated {updated_count} rows in column L and column M")
    print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
    print(f"❌ {len(row_map) - updated_count} rows could not be updated")

# Main execution
print("🚀 Starting to process all engine oil worksheets...")

# Create a global cache for bus worksheets to reuse across all worksheets
global_bus_worksheet_cache = {}

# Process each worksheet in sequence
for worksheet_name in engine_oil_worksheets:
    print(f"\n{'='*80}\n📊 Processing worksheet: {worksheet_name}\n{'='*80}")
    process_worksheet(worksheet_name)
    # Add a delay between worksheets to avoid rate limiting
    time.sleep(2)

print("\n✅ All worksheets have been processed successfully!")
print("💡 Performance Summary:")
print(f"✓ Processed {len(engine_oil_worksheets)} worksheets with batch processing")
print(f"✓ Used parallel processing for bus data with ThreadPoolExecutor")
print(f"✓ Implemented caching to reduce API calls")
print(f"✓ Used batch updates to minimize API requests")


In [ ]:
#FG_Working, Best Code

# Modified to process multiple engine oil worksheets, Added batch processing done

# Modified to process multiple engine oil worksheets with batch processing
# With fallback date display in M Column, For ATR_MAS_Engine Oil

# Make sure to include these imports at the top of your code
import re
from datetime import datetime
import random
import time
from collections import defaultdict
import gspread
import math
from concurrent.futures import ThreadPoolExecutor

# List of all worksheets to process
engine_oil_worksheets = [
    'engine_oil_CH_BSIV_CH_MAS_FG',
    'engine_oil_CK_BSVI_CK_MAS_FG'
]

# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

# Function to process each worksheet
def process_worksheet(worksheet_name):
    print(f"\n🔄 Processing worksheet: {worksheet_name}")

    # Get the worksheet object
    worksheet = None
    try:
        # Assuming these are defined in your notebook environment
        worksheet = eval(worksheet_name)  # This will use the already defined variable in your notebook
    except:
        print(f"❌ Error: Worksheet '{worksheet_name}' not found or not accessible")
        return

    # Clear data in range L2:M, Odometer value in Engine Oil and Fallback date removal
    clear_range = 'L2:M'
    print(f"🧹 Clearing range {clear_range} in {worksheet_name}...")
    worksheet.batch_clear([clear_range])

    print("🔍 Starting data processing with date fallback search...")

    # 📥 Load main sheet
    print("📥 Loading worksheet data...")
    main_data = worksheet.get_all_values()
    print(f"✅ Loaded {len(main_data)-1} rows from worksheet")

    # Step 1: Collect unique (bus, date) and row mapping
    print("🔢 Mapping bus numbers and dates...")
    bus_date_map = defaultdict(set)
    row_map = {}  # maps row number → (bus, date)

    for i, row in enumerate(main_data[1:], start=2):
        try:
            date_str = row[1].strip()
            bus_num = row[4].strip()
            if date_str and bus_num:
                bus_date_map[bus_num].add(date_str)
                row_map[i] = (bus_num, date_str)
        except:
            continue

    print(f"✅ Found {len(bus_date_map)} unique bus numbers")
    print(f"✅ Found {len(row_map)} rows to process")

    # Create a cache for worksheet data to avoid redundant fetches
    bus_worksheet_cache = {}

    # Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
    print("\n🔍 Searching for exact dates or closest earlier dates...")
    bus_date_value_map = {}
    match_type_map = {}  # Tracks whether each match is exact or fallback
    fallback_date_map = {}  # Store the actual fallback date used
    success_count = 0
    fallback_count = 0

    # Function to process a single bus
    def process_bus(bus_dates_tuple):
        bus, dates = bus_dates_tuple
        sheet_success = 0
        results = []

        try:
            # Check if we already have this bus's data in cache
            if bus in bus_worksheet_cache:
                sheet_data = bus_worksheet_cache[bus]
            else:
                # Fetch the bus worksheet data and store in cache
                sheet = Odometer_spreadsheet_FG.worksheet(bus)
                sheet_data = sheet.get_all_values()
                bus_worksheet_cache[bus] = sheet_data

            # Create a dictionary for faster exact date lookups
            date_value_lookup = {}
            for row in sheet_data[1:]:
                try:
                    if len(row) > 12:
                        date_str_key = row[1].strip()
                        date_value_lookup[date_str_key] = row[12]  # Column M
                except:
                    continue

            for date_str in dates:
                # Check for exact match using the lookup dictionary
                exact_value = date_value_lookup.get(date_str)

                if exact_value:
                    results.append((bus, date_str, exact_value, "exact", ""))
                    sheet_success += 1
                else:
                    # Try to find closest earlier date
                    value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                    if value:
                        results.append((bus, date_str, value, "fallback", fallback_date))
                        sheet_success += 1

            print(f"  ✅ Found {sheet_success}/{len(dates)} values for bus {bus}")
            return results

        except Exception as e:
            print(f"❌ Could not process sheet '{bus}': {str(e)}")
            return []

    # Process buses in parallel using ThreadPoolExecutor
    print(f"🚀 Processing {len(bus_date_map)} buses in parallel...")

    # Convert to list for ThreadPoolExecutor
    bus_dates_list = list(bus_date_map.items())

    # Use ThreadPoolExecutor for parallel processing
    # Adjust max_workers based on your system's capabilities
    with ThreadPoolExecutor(max_workers=5) as executor:
        all_results = list(executor.map(process_bus, bus_dates_list))

    # Process all results
    for bus_results in all_results:
        for bus, date_str, value, match_type, fallback_date in bus_results:
            bus_date_value_map[(bus, date_str)] = value
            match_type_map[(bus, date_str)] = match_type
            fallback_date_map[(bus, date_str)] = fallback_date

            if match_type == "exact":
                success_count += 1
            else:
                fallback_count += 1

    print(f"\n✅ Found values for {success_count} exact date matches")
    print(f"✅ Found values for {fallback_count} fallback date matches")
    print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

    # Step 3: Update values into Column L of main sheet and fallback dates into Column M
    print(f"\n📝 Preparing batch updates for {worksheet_name}...")

    # Prepare batch update for all cells at once
    batch_updates = []
    exact_match_cells = []
    fallback_match_cells = []

    for row_num, (bus, date_str) in row_map.items():
        value = bus_date_value_map.get((bus, date_str), "")
        fallback_date = fallback_date_map.get((bus, date_str), "")

        if value:
            # Add to batch updates list (row, col, value)
            batch_updates.append({
                'row': row_num,
                'col': 12,  # Column L
                'value': value
            })

            # Add fallback date update
            batch_updates.append({
                'row': row_num,
                'col': 13,  # Column M
                'value': fallback_date
            })

            # Track which cells need which formatting
            match_type = match_type_map.get((bus, date_str))
            if match_type == "exact":
                exact_match_cells.append(f"L{row_num}")
            elif match_type == "fallback":
                fallback_match_cells.append(f"L{row_num}")

    print(f"✅ Prepared {len(batch_updates)} cell updates")

    # Execute batch updates in chunks to avoid rate limits
    updated_count = 0
    chunk_size = 50  # Adjust based on API limits
    chunks = math.ceil(len(batch_updates) / chunk_size)

    print(f"🔄 Executing batch updates in {chunks} chunks...")

    for chunk_index in range(chunks):
        start_idx = chunk_index * chunk_size
        end_idx = min(start_idx + chunk_size, len(batch_updates))
        current_chunk = batch_updates[start_idx:end_idx]

        max_retries = 5
        retry_count = 0
        update_successful = False

        while not update_successful and retry_count < max_retries:
            try:
                # Prepare the batch update request
                cell_list = []

                for update in current_chunk:
                    cell = worksheet.cell(update['row'], update['col'])
                    cell.value = update['value']
                    cell_list.append(cell)

                # Execute the batch update
                worksheet.update_cells(cell_list, value_input_option='USER_ENTERED')

                update_successful = True
                updated_count += len(current_chunk) // 2  # Divide by 2 because we have 2 cells per row
                print(f"📊 Progress: Chunk {chunk_index+1}/{chunks} complete - {updated_count}/{len(batch_updates)//2} rows updated")

                # Add a small delay between chunks to avoid rate limits
                time.sleep(1)

            except Exception as e:
                retry_count += 1
                if "429" in str(e):
                    wait_time = (2 ** retry_count) + random.random()
                    print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Failed to update chunk {chunk_index+1}: {e}")
                    break

        if not update_successful:
            print(f"❌ Failed to update chunk {chunk_index+1} after {max_retries} retries")

    # Apply formatting in batches (currently commented out)
    print("\n🎨 Applying formatting to distinguish match types...")
    # You can uncomment and implement batch formatting logic if needed.

    print(f"\n✅ COMPLETE for {worksheet_name}: Updated {updated_count} rows in column L and column M")
    print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
    print(f"❌ {len(row_map) - updated_count} rows could not be updated")

# Main execution
print("🚀 Starting to process all engine oil worksheets...")

# Create a global cache for bus worksheets to reuse across all worksheets
global_bus_worksheet_cache = {}

# Process each worksheet in sequence
for worksheet_name in engine_oil_worksheets:
    print(f"\n{'='*80}\n📊 Processing worksheet: {worksheet_name}\n{'='*80}")
    process_worksheet(worksheet_name)
    # Add a delay between worksheets to avoid rate limiting
    time.sleep(2)

print("\n✅ All worksheets have been processed successfully!")
print("💡 Performance Summary:")
print(f"✓ Processed {len(engine_oil_worksheets)} worksheets with batch processing")
print(f"✓ Used parallel processing for bus data with ThreadPoolExecutor")
print(f"✓ Implemented caching to reduce API calls")
print(f"✓ Used batch updates to minimize API requests")


In [ ]:
# Lean for all units, improved API limiter, #Lean combined all Units Engine Oil and Search all Odometer, but it search only missing values, if it works, then in future just add engine oil sheet and odometer sheet


# Lean version - Only processes rows with empty L column
# With fallback date display in M Column, For SVP, FG, BT, RT, MB, and DP

import re
from datetime import datetime
import random
import time
from collections import defaultdict
import gspread
import math
from concurrent.futures import ThreadPoolExecutor

# List of all worksheets to process
engine_oil_worksheets = [
    'engine_oil_CH_BSIV_CH_MAS_FG',
    'engine_oil_CK_BSVI_CK_MAS_FG',
    'engine_oil_CH_BSIV_CH_MAS_ATR',
    'engine_oil_CK_BSVI_CK_MAS_ATR',
    'engine_oil_CH_BSIV_CH_MAS_rr',
    'engine_oil_CK_BSVI_CK_MAS_rr'
]

# List of all odometer spreadsheets to search
odometer_spreadsheets = [
    'Odometer_spreadsheet_SVP',
    'Odometer_spreadsheet_FG',
    'Odometer_spreadsheet_BT',
    'Odometer_spreadsheet_RT',
    'Odometer_spreadsheet_MB',
    'Odometer_spreadsheet_DP'
]

# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

# Global cache for bus worksheets to reuse across all worksheets and spreadsheets
global_bus_worksheet_cache = {}

# Function to process each worksheet - LEAN VERSION (only empty L cells)
def process_worksheet_lean(worksheet_name):
    print(f"\n🔄 Processing worksheet: {worksheet_name}")

    # Get the worksheet object
    worksheet = None
    try:
        # Assuming these are defined in your notebook environment
        worksheet = eval(worksheet_name)  # This will use the already defined variable in your notebook
    except:
        print(f"❌ Error: Worksheet '{worksheet_name}' not found or not accessible")
        return

    # NO CLEARING OF RANGES - we're only updating empty cells
    print("🔍 Starting data processing for EMPTY L cells only...")

    # 📥 Load main sheet
    print("📥 Loading worksheet data...")
    main_data = worksheet.get_all_values()
    print(f"✅ Loaded {len(main_data)-1} rows from worksheet")

    # Step 1: Collect unique (bus, date) and row mapping ONLY FOR EMPTY L CELLS
    print("🔢 Mapping bus numbers and dates for empty L cells...")
    bus_date_map = defaultdict(set)
    row_map = {}  # maps row number → (bus, date)
    empty_cell_count = 0

    for i, row in enumerate(main_data[1:], start=2):
        try:
            # Check if L column (index 11) is empty
            l_value = row[11].strip() if len(row) > 11 else ""

            if not l_value:  # Only process if L column is empty
                date_str = row[1].strip()
                bus_num = row[4].strip()
                if date_str and bus_num:
                    bus_date_map[bus_num].add(date_str)
                    row_map[i] = (bus_num, date_str)
                    empty_cell_count += 1
        except:
            continue

    print(f"✅ Found {len(bus_date_map)} unique bus numbers with empty L cells")
    print(f"✅ Found {empty_cell_count} empty L cells to process")

    # Exit early if no empty cells to process
    if empty_cell_count == 0:
        print(f"✓ No empty L cells to process in {worksheet_name}. Moving to next worksheet.")
        return

    # Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
    print("\n🔍 Searching for exact dates or closest earlier dates...")
    bus_date_value_map = {}
    match_type_map = {}  # Tracks whether each match is exact or fallback
    fallback_date_map = {}  # Store the actual fallback date used
    success_count = 0
    fallback_count = 0

    # Track which spreadsheet contained the match for metadata
    spreadsheet_source_map = {}  # Maps (bus, date) to spreadsheet name

    # Function to process a single bus across all spreadsheets
    def process_bus(bus_dates_tuple):
        bus, dates = bus_dates_tuple
        sheet_success = 0
        results = []
        dates_to_process = set(dates)  # Create a copy to safely modify

        # Try each spreadsheet in sequence
        for spreadsheet_name in odometer_spreadsheets:
            if not dates_to_process:  # If all dates have been found, skip further processing
                break

            try:
                # Generate cache key that includes the spreadsheet name
                cache_key = f"{spreadsheet_name}:{bus}"

                # Check if we already have this bus's data in cache
                if cache_key in global_bus_worksheet_cache:
                    sheet_data = global_bus_worksheet_cache[cache_key]
                else:
                    # Get the spreadsheet object dynamically
                    try:
                        spreadsheet = eval(spreadsheet_name)
                        # Try to get the worksheet for this bus with retry mechanism
                        max_retries = 5
                        retry_count = 0
                        while retry_count < max_retries:
                            try:
                                sheet = spreadsheet.worksheet(bus)
                                sheet_data = sheet.get_all_values()
                                global_bus_worksheet_cache[cache_key] = sheet_data
                                break  # Success, exit retry loop
                            except gspread.exceptions.APIError as api_error:
                                if hasattr(api_error, 'response') and api_error.response.status_code == 429:
                                    retry_count += 1
                                    wait_time = (2 ** retry_count) + (random.random() * 2)
                                    print(f"⏳ Rate limit hit for bus {bus}, waiting {wait_time:.2f}s (retry {retry_count}/{max_retries})")
                                    time.sleep(wait_time)
                                    if retry_count == max_retries:
                                        print(f"⚠️ Max retries reached for bus {bus} in {spreadsheet_name}")
                                        raise
                                else:
                                    # Not a rate limit issue, the worksheet likely doesn't exist
                                    raise
                            except Exception as e:
                                # The worksheet doesn't exist in this spreadsheet
                                if "Worksheet not found" in str(e) or "404" in str(e):
                                    print(f"  ℹ️ Bus {bus} not found in {spreadsheet_name}")
                                else:
                                    print(f"  ⚠️ Error accessing {bus} in {spreadsheet_name}: {str(e)}")
                                raise
                    except Exception:
                        # This bus doesn't exist in this spreadsheet or we couldn't access it
                        continue

                # Process each date for this bus in this spreadsheet
                # Create a dictionary for faster exact date lookups
                date_value_lookup = {}
                for row in sheet_data[1:]:
                    try:
                        if len(row) > 12:
                            date_str_key = row[1].strip()
                            date_value_lookup[date_str_key] = row[12]  # Column M
                    except:
                        continue

                dates_found_in_this_sheet = set()

                # First try exact matches which are faster to check
                for date_str in dates_to_process:
                    # Check for exact match using the lookup dictionary
                    exact_value = date_value_lookup.get(date_str)
                    if exact_value:
                        results.append((bus, date_str, exact_value, "exact", "", spreadsheet_name))
                        sheet_success += 1
                        dates_found_in_this_sheet.add(date_str)

                # Then try fallback matches for remaining dates
                remaining_dates = dates_to_process - dates_found_in_this_sheet
                for date_str in remaining_dates:
                    # Try to find closest earlier date
                    value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                    if value:
                        results.append((bus, date_str, value, "fallback", fallback_date, spreadsheet_name))
                        sheet_success += 1
                        dates_found_in_this_sheet.add(date_str)

                # Remove all processed dates
                dates_to_process -= dates_found_in_this_sheet

            except Exception as e:
                if "Worksheet not found" not in str(e) and "404" not in str(e):
                    print(f"❌ Could not process sheet '{bus}' in spreadsheet '{spreadsheet_name}': {str(e)}")
                continue

        if sheet_success > 0:
            print(f"  ✅ Found {sheet_success} values for bus {bus}")
        else:
            print(f"  ⚠️ No values found for bus {bus} in any spreadsheet")

        return results

    # Only process if we have buses to check
    if len(bus_date_map) > 0:
        # Convert to list for ThreadPoolExecutor
        bus_dates_list = list(bus_date_map.items())

        # Use ThreadPoolExecutor for parallel processing
        max_workers = min(5, len(bus_dates_list))  # Adjust based on data size
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            all_results = list(executor.map(process_bus, bus_dates_list))

        # Process all results
        for bus_results in all_results:
            for bus, date_str, value, match_type, fallback_date, source_spreadsheet in bus_results:
                bus_date_value_map[(bus, date_str)] = value
                match_type_map[(bus, date_str)] = match_type
                fallback_date_map[(bus, date_str)] = fallback_date
                spreadsheet_source_map[(bus, date_str)] = source_spreadsheet

                if match_type == "exact":
                    success_count += 1
                else:
                    fallback_count += 1
    else:
        print("⚠️ No buses to process - all L cells have values already")

    print(f"\n✅ Found values for {success_count} exact date matches")
    print(f"✅ Found values for {fallback_count} fallback date matches")
    print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

    # Step 3: Update values into Column L of main sheet and fallback dates into Column M
    print(f"\n📝 Preparing batch updates for {worksheet_name}...")

    # Prepare batch update for all cells at once
    batch_updates = []
    exact_match_cells = []
    fallback_match_cells = []

    for row_num, (bus, date_str) in row_map.items():
        value = bus_date_value_map.get((bus, date_str), "")
        fallback_date = fallback_date_map.get((bus, date_str), "")
        spreadsheet_source = spreadsheet_source_map.get((bus, date_str), "")

        if value:
            # Add to batch updates list (row, col, value)
            batch_updates.append({
                'row': row_num,
                'col': 12,  # Column L
                'value': value
            })

            # Add fallback date update
            batch_updates.append({
                'row': row_num,
                'col': 13,  # Column M
                'value': fallback_date if fallback_date else ""
            })

            # Add source spreadsheet info in Column N for tracking
     #       batch_updates.append({
      #          'row': row_num,
      #          'col': 14,  # Column N
      #          'value': spreadsheet_source
      #      })

            # Track which cells need which formatting
            match_type = match_type_map.get((bus, date_str))
            if match_type == "exact":
                exact_match_cells.append(f"L{row_num}")
            elif match_type == "fallback":
                fallback_match_cells.append(f"L{row_num}")

    print(f"✅ Prepared {len(batch_updates)} cell updates")

    # Skip if no updates needed
    if len(batch_updates) == 0:
        print("✓ No updates needed for this worksheet")
        return

    # Execute batch updates in chunks to avoid rate limits
    updated_count = 0
    chunk_size = 25  # Reduced chunk size to avoid API limits
    chunks = math.ceil(len(batch_updates) / chunk_size)

    print(f"🔄 Executing batch updates in {chunks} chunks...")

    for chunk_index in range(chunks):
        start_idx = chunk_index * chunk_size
        end_idx = min(start_idx + chunk_size, len(batch_updates))
        current_chunk = batch_updates[start_idx:end_idx]

        max_retries = 8  # Increased retries
        retry_count = 0
        update_successful = False

        while not update_successful and retry_count < max_retries:
            try:
                # Prepare the batch update request
                cell_list = []

                for update in current_chunk:
                    cell = worksheet.cell(update['row'], update['col'])
                    cell.value = update['value']
                    cell_list.append(cell)

                # Execute the batch update using our rate-limited function
                def do_update():
                    return worksheet.update_cells(cell_list, value_input_option='USER_ENTERED')

                rate_limited_request(do_update)

                update_successful = True
                updated_count += len(current_chunk) // 3  # Divide by 3 because we have 3 cells per row (L, M, N)
                print(f"📊 Progress: Chunk {chunk_index+1}/{chunks} complete - {updated_count}/{len(batch_updates)//3} rows updated")

                # Add a variable delay between chunks to avoid rate limits
                # More aggressive backoff with increasing chunk index
                wait_time = 2 + (chunk_index % 3) + (random.random() * 3)
                print(f"⏳ Waiting {wait_time:.2f}s before next chunk...")
                time.sleep(wait_time)

            except gspread.exceptions.APIError as api_error:
                retry_count += 1
                if hasattr(api_error, 'response') and api_error.response.status_code == 429:
                    # Exponential backoff with jitter for rate limits
                    wait_time = (2 ** retry_count) + (random.random() * 5)
                    # Cap maximum wait time at 2 minutes
                    wait_time = min(wait_time, 120)
                    print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ API Error on chunk {chunk_index+1}: {api_error}")
                    if retry_count < 3:  # Only retry a few times for non-rate-limit errors
                        time.sleep(5)
                    else:
                        break
            except Exception as e:
                retry_count += 1
                print(f"❌ Failed to update chunk {chunk_index+1}: {e}")
                if retry_count < 3:  # Only retry a few times for general errors
                    time.sleep(5)
                else:
                    break

        if not update_successful:
            print(f"❌ Failed to update chunk {chunk_index+1} after {max_retries} retries")
            # Add extra delay when a chunk completely fails before moving to next chunk
            wait_time = 15 + (random.random() * 10)
            print(f"⏳ Taking a longer break: {wait_time:.2f}s before continuing...")
            time.sleep(wait_time)

    # Apply formatting in batches (currently commented out)
    print("\n🎨 Applying formatting to distinguish match types...")
    # You can uncomment and implement batch formatting logic if needed.

    print(f"\n✅ COMPLETE for {worksheet_name}: Updated {updated_count} rows in columns L, M, and N")
    print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
    print(f"❌ {len(row_map) - updated_count} rows could not be updated")

# Global rate limiting function
def rate_limited_request(func, *args, **kwargs):
    """Execute a function with rate limiting and retries for API errors"""
    max_retries = 8  # More retries for critical operations
    retry_count = 0
    base_wait_time = 2  # seconds

    while retry_count < max_retries:
        try:
            return func(*args, **kwargs)
        except gspread.exceptions.APIError as api_error:
            if hasattr(api_error, 'response') and api_error.response.status_code == 429:
                retry_count += 1
                # Exponential backoff with jitter
                wait_time = (base_wait_time ** retry_count) + (random.random() * 5)
                # Cap the wait time at 3 minutes
                wait_time = min(wait_time, 180)
                print(f"⏳ Rate limit hit! Waiting {wait_time:.2f}s (retry {retry_count}/{max_retries})")
                time.sleep(wait_time)
            else:
                # Re-raise non-rate-limit errors
                raise
        except Exception as e:
            # For other exceptions, retry a few times but with less patience
            if retry_count < 3:
                retry_count += 1
                wait_time = base_wait_time + (random.random() * 2)
                print(f"⚠️ Error occurred: {str(e)}")
                print(f"Retrying in {wait_time:.2f}s (retry {retry_count}/3)")
                time.sleep(wait_time)
            else:
                raise

    # If we've exhausted all retries
    raise Exception(f"Failed after {max_retries} retries due to persistent rate limiting")

# Main execution - LEAN VERSION with improved rate limiting
print("🚀 Starting LEAN processing - only empty L cells...")
print(f"🔍 Will search across {len(odometer_spreadsheets)} odometer spreadsheets: {', '.join(odometer_spreadsheets)}")

# Process each worksheet with rate limiting and proper delays
for i, worksheet_name in enumerate(engine_oil_worksheets):
    print(f"\n{'='*80}\n📊 Processing worksheet: {worksheet_name} ({i+1}/{len(engine_oil_worksheets)})\n{'='*80}")

    # Add a longer delay between worksheets to avoid rate limiting
    if i > 0:
        wait_time = 5 + (random.random() * 5)  # 5-10 second delay between worksheets
        print(f"⏳ Waiting {wait_time:.2f}s before processing next worksheet...")
        time.sleep(wait_time)

    # Process the worksheet with proper rate limiting
    try:
        process_worksheet_lean(worksheet_name)
    except Exception as e:
        print(f"❌ Error processing worksheet {worksheet_name}: {str(e)}")
        print("Continuing with next worksheet after a delay...")
        time.sleep(15)  # Longer delay after an error

print("\n✅ All worksheets have been processed successfully!")
print("💡 Performance Summary:")
print(f"✓ Processed {len(engine_oil_worksheets)} worksheets with LEAN mode (empty L cells only)")
print(f"✓ Searched across {len(odometer_spreadsheets)} odometer spreadsheets")
print(f"✓ Used parallel processing for bus data with ThreadPoolExecutor")
print(f"✓ Implemented caching to reduce API calls")
print(f"✓ Used batch updates to minimize API requests")



🚀 Starting LEAN processing - only empty L cells...
🔍 Will search across 6 odometer spreadsheets: Odometer_spreadsheet_SVP, Odometer_spreadsheet_FG, Odometer_spreadsheet_BT, Odometer_spreadsheet_RT, Odometer_spreadsheet_MB, Odometer_spreadsheet_DP

📊 Processing worksheet: engine_oil_CH_BSIV_CH_MAS_FG (1/6)

🔄 Processing worksheet: engine_oil_CH_BSIV_CH_MAS_FG
🔍 Starting data processing for EMPTY L cells only...
📥 Loading worksheet data...
✅ Loaded 999 rows from worksheet
🔢 Mapping bus numbers and dates for empty L cells...
✅ Found 23 unique bus numbers with empty L cells
✅ Found 489 empty L cells to process

🔍 Searching for exact dates or closest earlier dates...
  ⚠️ Error accessing 388 in Odometer_spreadsheet_SVP: 388
  ⚠️ Error accessing 389 in Odometer_spreadsheet_SVP: 389
  ⚠️ Error accessing 415 in Odometer_spreadsheet_SVP: 415
  ⚠️ Error accessing 388 in Odometer_spreadsheet_FG: 388
  ⚠️ Error accessing tyreplant in Odometer_spreadsheet_SVP: tyreplant
  ⚠️ Error accessing 389 in 

In [ ]:
# # High Speed, Lean for all units, improved API limiter, #Lean combined all Units Engine Oil and Search all Odometer, but it search only missing values, if it works, then in future just add engine oil sheet and odometer sheet



# The above code is returning value in Column N, i dont need it, also incorporate batch processing to increase speed

In [ ]:
# If date is current date then it is returning the spreadsheet name, trying to correct, It is wokring, i will delete above code in version 1.0.9

# High speed Report Generation, in Engine Oil Report Spreadsheet

from datetime import datetime
import re
import time
from collections import defaultdict
import functools
import concurrent.futures
import threading

# === PERFORMANCE OPTIMIZATION SETUP ===
print("🚀 Initializing performance optimizations...")

# Cache for worksheet data to reduce API calls
data_cache = {}
cache_lock = threading.Lock()

# LRU Cache decorator for expensive operations
def lru_cache_with_expiry(maxsize=128, typed=False, ttl=600):
    """LRU Cache with expiry time"""
    def decorator(func):
        cache_dict = {}
        cache_times = {}
        cache_lock = threading.Lock()

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = str(args) + str(kwargs)
            current_time = time.time()

            with cache_lock:
                # Check if key exists and not expired
                if key in cache_dict and current_time - cache_times[key] < ttl:
                    return cache_dict[key]

                # Call the function and cache result
                result = func(*args, **kwargs)
                cache_dict[key] = result
                cache_times[key] = current_time

                # Remove oldest entries if cache is too large
                if len(cache_dict) > maxsize:
                    oldest_key = min(cache_times, key=cache_times.get)
                    cache_dict.pop(oldest_key)
                    cache_times.pop(oldest_key)

                return result
        return wrapper
    return decorator

# Batch processing helper functions
def chunk_list(lst, chunk_size):
    """Split list into chunks of specified size"""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

# Cached function to get worksheet data
@lru_cache_with_expiry(maxsize=100, ttl=300)  # Cache for 5 minutes
def get_worksheet_data(spreadsheet_name, worksheet_name):
    """Get worksheet data with caching"""
    cache_key = f"{spreadsheet_name}:{worksheet_name}"

    with cache_lock:
        if cache_key in data_cache:
            print(f"📋 Cache hit for {cache_key}")
            return data_cache[cache_key]

    # Function to find the actual worksheet object
    def find_worksheet(spreadsheet_list, worksheet_name, spreadsheet_names):
        for idx, spreadsheet in enumerate(spreadsheet_list):
            try:
                sheet = spreadsheet.worksheet(worksheet_name)
                print(f"📑 Found worksheet {worksheet_name} in {spreadsheet_names[idx]}")
                return sheet, spreadsheet_names[idx]
            except Exception:
                continue
        return None, None

    # Cache miss, need to retrieve data
    try:
        sheet, source = find_worksheet(odometer_spreadsheets, worksheet_name, spreadsheet_names)
        if sheet:
            data = sheet.get_all_values()

            with cache_lock:
                data_cache[cache_key] = (data, source)

            return data, source
        return None, None
    except Exception as e:
        print(f"⚠️ Error retrieving data for {worksheet_name}: {e}")
        return None, None

# === STEP 1: CLEAR DATA ===
print("\n🧹 STEP 1: Clearing data...")
clear_range_all_OdometersheetID = 'A5:R200'
clear_range_all_Report_EO1_Worksheet = ['B2:B200', 'AC2:AC200']

OdometersheetID.batch_clear([clear_range_all_OdometersheetID])
Report_EO1_Worksheet.batch_clear(clear_range_all_Report_EO1_Worksheet)

# === STEP 2: PROCESS BUS DATA ===
print("\n📊 STEP 2: Processing bus data...")

# Function to process spreadsheet data in parallel
def process_spreadsheet(cell_ref, position_index):
    """Process a single spreadsheet and return its data"""
    odometer_id = OdometersheetID.acell(cell_ref).value
    spreadsheet = gc.open_by_key(odometer_id)
    bus_numbers = sorted([worksheet.title for worksheet in spreadsheet])
    spreadsheet_name = spreadsheet.title
    return {
        'spreadsheet': spreadsheet,
        'name': spreadsheet_name,
        'buses': bus_numbers,
        'position': position_index
    }

# Process all spreadsheets in parallel
spreadsheet_info = {
    'SVP': {'cell': 'A2', 'position': 0},
    'FG': {'cell': 'B2', 'position': 1},
    'BT': {'cell': 'C2', 'position': 2},
    'RT': {'cell': 'D2', 'position': 3},
    'MB': {'cell': 'E2', 'position': 4},
    'DP': {'cell': 'F2', 'position': 5}
}

spreadsheet_data = {}
odometer_spreadsheets = [None] * 6
spreadsheet_names = [None] * 6

# For Google Sheets API, we can't fully parallelize due to rate limits
# But we can optimize the processing steps
for key, info in spreadsheet_info.items():
    data = process_spreadsheet(info['cell'], info['position'])
    spreadsheet_data[key] = data
    odometer_spreadsheets[info['position']] = data['spreadsheet']
    spreadsheet_names[info['position']] = data['name']
    print(f"✅ Processed {key} spreadsheet: {data['name']} with {len(data['buses'])} buses")

# Calculate starting rows for each section
start_row_svp = 2
start_row_fg = start_row_svp + len(spreadsheet_data['SVP']['buses'])
start_row_bt = start_row_fg + len(spreadsheet_data['FG']['buses'])
start_row_rt = start_row_bt + len(spreadsheet_data['BT']['buses'])
start_row_mb = start_row_rt + len(spreadsheet_data['RT']['buses'])
start_row_dp = start_row_mb + len(spreadsheet_data['MB']['buses'])

# === STEP 3: UPDATE ODOMETERSHEETID WITH BUS NUMBERS ===
print("\n📝 STEP 3: Updating OdometersheetID with bus numbers...")

# Optimized batch update function
def optimized_batch_update(worksheet, updates):
    """Process updates in optimal batch sizes"""
    BATCH_SIZE = 100  # Adjust based on API limits

    # Split updates into manageable chunks
    chunked_updates = chunk_list(updates, BATCH_SIZE)

    for chunk in chunked_updates:
        try:
            worksheet.batch_update(chunk)
            time.sleep(0.5)  # Slight delay to avoid rate limiting
        except Exception as e:
            print(f"⚠️ Batch update error: {e}")
            # Fall back to smaller chunks if needed
            for update in chunk:
                try:
                    worksheet.update(update['range'], update['values'])
                    time.sleep(0.1)
                except Exception as e2:
                    print(f"⚠️ Individual update error at {update['range']}: {e2}")

# Update spreadsheet names in row 4 (batch update)
name_updates = [
    {'range': 'A4', 'values': [[spreadsheet_data['SVP']['name']]]},
    {'range': 'B4', 'values': [[spreadsheet_data['FG']['name']]]},
    {'range': 'C4', 'values': [[spreadsheet_data['BT']['name']]]},
    {'range': 'D4', 'values': [[spreadsheet_data['RT']['name']]]},
    {'range': 'E4', 'values': [[spreadsheet_data['MB']['name']]]},
    {'range': 'F4', 'values': [[spreadsheet_data['DP']['name']]]}
]
OdometersheetID.batch_update(name_updates)
print("✅ Updated all spreadsheet names in row 4")

# Generate all updates for bus numbers
all_bus_updates = []

# Helper function to create batch updates for a column
def create_column_updates(column_letter, start_row, buses):
    updates = []
    for i, bus in enumerate(buses):
        row = start_row + i
        # Try to convert numeric strings to integers
        try:
            if isinstance(bus, str) and bus.isdigit():
                bus = int(bus)
        except:
            pass
        updates.append({
            'range': f'{column_letter}{row}',
            'values': [[bus]]
        })
    return updates

# Create all updates for bus numbers
all_bus_updates.extend(create_column_updates('A', 5, spreadsheet_data['SVP']['buses']))
all_bus_updates.extend(create_column_updates('B', 5, spreadsheet_data['FG']['buses']))
all_bus_updates.extend(create_column_updates('C', 5, spreadsheet_data['BT']['buses']))
all_bus_updates.extend(create_column_updates('D', 5, spreadsheet_data['RT']['buses']))
all_bus_updates.extend(create_column_updates('E', 5, spreadsheet_data['MB']['buses']))
all_bus_updates.extend(create_column_updates('F', 5, spreadsheet_data['DP']['buses']))

# Batch update OdometersheetID with bus numbers
optimized_batch_update(OdometersheetID, all_bus_updates)
print(f"✅ Updated {len(all_bus_updates)} bus entries in OdometersheetID")

# === STEP 4: UPDATE REPORT_EO1 WORKSHEET WITH ALL BUS NUMBERS ===
print("\n📊 STEP 4: Updating Report_EO1 with all bus numbers...")

# Combine all bus lists for ReportA_EO1
all_buses_with_sources = []
for key, data in spreadsheet_data.items():
    for bus in data['buses']:
        all_buses_with_sources.append((bus, data['name']))

# Generate batch updates for Report_EO1
report_EO1_updates = []
for i, (bus, source) in enumerate(all_buses_with_sources):
    row = i + 2  # Start from row 2
    # Try to convert numeric strings to integers
    try:
        if isinstance(bus, str) and bus.isdigit():
            bus = int(bus)
    except:
        pass

    report_EO1_updates.append({
        'range': f'B{row}',
        'values': [[bus]]
    })
    report_EO1_updates.append({
        'range': f'AC{row}',
        'values': [[source]]
    })

# Batch update Report_EO1
optimized_batch_update(Report_EO1_Worksheet, report_EO1_updates)
print(f"✅ Updated {len(all_buses_with_sources)} bus entries in Report_EO1 worksheet")

print("🎉 Bus data preparation completed successfully!")

# === STEP 5: SEARCH FOR ODOMETER VALUES ===
print("\n🔍 STEP 5: Searching for odometer values...")

# Clear columns C and D
Report_EO1_Worksheet.batch_clear(["C2:D"])

# Get current date in expected format (e.g., "05,May25")
current_date_str = datetime.now().strftime('%d,%b%y')

# Get main data from Report_EO1
main_data = Report_EO1_Worksheet.get_all_values()
header, rows = main_data[0], main_data[1:]

# Map each row to bus + date (using today's date)
row_map = {}
bus_set = set()
for i, row in enumerate(rows, start=2):
    try:
        if len(row) > 1:  # Make sure row has at least 2 columns
            bus = re.sub(r'[^a-zA-Z0-9]', '', str(row[1]).strip()).upper()  # Clean bus name
            if bus:
                row_map[i] = (bus, current_date_str)
                bus_set.add(bus)
    except Exception as e:
        print(f"⚠️ Error processing row {i}: {e}")
        continue

print(f"🔎 Total buses to process: {len(bus_set)}")
if bus_set:
    print("Sample buses:", list(bus_set)[:5])  # Log first 5 buses

# Initialize lookup maps with thread safety
bus_date_value_map = {}
match_type_map = {}
fallback_date_map = {}
source_spreadsheet_map = {}
map_lock = threading.Lock()

# Function to find closest earlier date value
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    """Find closest earlier date with a value in the specified column"""
    if not sheet_data:
        return None, ""

    try:
        target_date = datetime.strptime(target_date_str, '%d,%b%y')
        valid_rows = []

        for row in sheet_data[1:]:  # Skip header
            if len(row) <= value_column_index:
                continue

            date_cell = row[1].strip() if len(row) > 1 else ""
            value_cell = row[value_column_index].strip() if len(row) > value_column_index else ""

            if not date_cell or not value_cell:
                continue

            # Try multiple date formats
            parsed_date = None
            for fmt in ['%d,%b%y', '%d-%b-%y', '%d/%b/%y']:
                try:
                    parsed_date = datetime.strptime(date_cell, fmt)
                    break
                except:
                    continue

            if parsed_date and parsed_date <= target_date and value_cell:
                valid_rows.append((parsed_date, date_cell, value_cell))

        if valid_rows:
            valid_rows.sort(key=lambda x: x[0], reverse=True)  # Newest date first
            best = valid_rows[0]
            return best[2], best[1]  # value, fallback_date

    except Exception as e:
        print(f"⚠️ Error in fallback for {target_date_str}: {e}")

    return None, ""

# Process a single bus to find odometer value
def process_bus_odometer(bus):
    """Process odometer data for a single bus"""
    found = False

    # Use the cached function to get worksheet data
    data, source = get_worksheet_data("All", bus)

    if data:
        found = True
        # Check for exact date match
        exact_match = False
        for row in data[1:]:  # Skip header
            if len(row) > 12 and row[1].strip() == current_date_str and row[12].strip():
                odometer_val = row[12].strip()
                if odometer_val and odometer_val.isdigit():  # Validate numeric
                    with map_lock:
                        bus_date_value_map[(bus, current_date_str)] = odometer_val
                        match_type_map[(bus, current_date_str)] = "exact"
                        fallback_date_map[(bus, current_date_str)] = current_date_str  # Store current date
                        source_spreadsheet_map[(bus, current_date_str)] = source
                    exact_match = True
                    print(f"📌 Bus {bus}: Exact match: {odometer_val} km on {current_date_str}")
                    break

        if not exact_match:
            # Fallback to closest date
            val, fallback_date = find_closest_date_value(data, current_date_str, 12)
            if val:
                with map_lock:
                    bus_date_value_map[(bus, current_date_str)] = val
                    match_type_map[(bus, current_date_str)] = "fallback"
                    fallback_date_map[(bus, current_date_str)] = fallback_date
                    source_spreadsheet_map[(bus, current_date_str)] = source
                print(f"🔄 Bus {bus}: Fallback match: {val} km on {fallback_date}")

    if not found:
        print(f"🚨 Bus {bus} not found in ANY spreadsheet!")

    return found

# Process buses in parallel batches
MAX_WORKERS = 5  # Adjust based on API limits
BATCH_SIZE = 10  # Process this many buses at once

def process_buses_in_batches(buses):
    """Process buses in parallel batches"""
    total = len(buses)
    completed = 0

    bus_chunks = chunk_list(list(buses), BATCH_SIZE)

    for chunk in bus_chunks:
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(chunk))) as executor:
            futures = {executor.submit(process_bus_odometer, bus): bus for bus in chunk}
            for future in concurrent.futures.as_completed(futures):
                bus = futures[future]
                try:
                    result = future.result()
                    completed += 1
                    if completed % 10 == 0 or completed == total:
                        print(f"Progress: {completed}/{total} buses processed ({completed/total:.1%})")
                except Exception as e:
                    print(f"⚠️ Error processing bus {bus}: {e}")

        # Small delay between batches to avoid API rate limits
        time.sleep(1)

    print(f"✅ Completed processing all {total} buses")

# Process all buses in batches
if bus_set:
    process_buses_in_batches(bus_set)

# Update Report_EO1 with logging
print("\n📝 Writing data to Report_EO1...")
updated_rows = []
for row_idx, (bus, date_str) in row_map.items():
    key = (bus, date_str)
    val = bus_date_value_map.get(key, "")
    fallback = fallback_date_map.get(key, "")
    match_type = match_type_map.get(key, "")
    source = source_spreadsheet_map.get(key, "")

    # Improved logic for Column C (date) and Column D (value)
    if match_type == "exact":
        # For exact matches, use the current date (date_str)
        col_c = date_str
        col_d = val if val else ""
    elif match_type == "fallback":
        # For fallback matches, use the fallback date we found
        col_c = fallback if fallback else ""
        col_d = val if val else ""
    else:
        # No match found
        col_c = ""
        col_d = ""

    # Add to batch update
    updated_rows.append([col_c, col_d])

# Prepare batch update
if updated_rows:
    # Split into manageable chunks
    MAX_ROWS_PER_UPDATE = 1000  # Adjust based on API limits
    row_chunks = chunk_list(updated_rows, MAX_ROWS_PER_UPDATE)

    for i, chunk in enumerate(row_chunks):
        start_row = 2 + i * MAX_ROWS_PER_UPDATE
        end_row = start_row + len(chunk) - 1
        range_to_update = f"C{start_row}:D{end_row}"

        Report_EO1_Worksheet.update(range_to_update, chunk)
        print(f"✅ Updated rows {start_row}-{end_row} in Report_EO1")

        # Small delay between batch updates
        if i < len(row_chunks) - 1:
            time.sleep(1)

    print(f"✅ All {len(updated_rows)} rows updated successfully in Report_EO1!")
else:
    print("❌ No updates made to Report_EO1.")

print("🎉 Complete script execution finished successfully!")




In [9]:
# Here i will try to get the HSD balance on current date


from datetime import datetime
import re
import time
from collections import defaultdict
import functools
import concurrent.futures
import threading

# === PERFORMANCE OPTIMIZATION SETUP ===
print("🚀 Initializing performance optimizations...")

# Cache for worksheet data to reduce API calls
data_cache = {}
cache_lock = threading.Lock()

# LRU Cache decorator for expensive operations
def lru_cache_with_expiry(maxsize=128, typed=False, ttl=600):
    """LRU Cache with expiry time"""
    def decorator(func):
        cache_dict = {}
        cache_times = {}
        cache_lock = threading.Lock()

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = str(args) + str(kwargs)
            current_time = time.time()

            with cache_lock:
                # Check if key exists and not expired
                if key in cache_dict and current_time - cache_times[key] < ttl:
                    return cache_dict[key]

                # Call the function and cache result
                result = func(*args, **kwargs)
                cache_dict[key] = result
                cache_times[key] = current_time

                # Remove oldest entries if cache is too large
                if len(cache_dict) > maxsize:
                    oldest_key = min(cache_times, key=cache_times.get)
                    cache_dict.pop(oldest_key)
                    cache_times.pop(oldest_key)

                return result
        return wrapper
    return decorator

# Batch processing helper functions
def chunk_list(lst, chunk_size):
    """Split list into chunks of specified size"""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

# Cached function to get worksheet data
@lru_cache_with_expiry(maxsize=100, ttl=300)  # Cache for 5 minutes
def get_worksheet_data(spreadsheet_name, worksheet_name):
    """Get worksheet data with caching"""
    cache_key = f"{spreadsheet_name}:{worksheet_name}"

    with cache_lock:
        if cache_key in data_cache:
            print(f"📋 Cache hit for {cache_key}")
            return data_cache[cache_key]

    # Function to find the actual worksheet object
    def find_worksheet(spreadsheet_list, worksheet_name, spreadsheet_names):
        for idx, spreadsheet in enumerate(spreadsheet_list):
            try:
                sheet = spreadsheet.worksheet(worksheet_name)
                print(f"📑 Found worksheet {worksheet_name} in {spreadsheet_names[idx]}")
                return sheet, spreadsheet_names[idx]
            except Exception:
                continue
        return None, None

    # Cache miss, need to retrieve data
    try:
        sheet, source = find_worksheet(odometer_spreadsheets, worksheet_name, spreadsheet_names)
        if sheet:
            data = sheet.get_all_values()

            with cache_lock:
                data_cache[cache_key] = (data, source)

            return data, source
        return None, None
    except Exception as e:
        print(f"⚠️ Error retrieving data for {worksheet_name}: {e}")
        return None, None

# === STEP 1: CLEAR DATA HSD from AF Coloumn
print("\n🧹 STEP 1: Clearing data...")


clear_range_HSD_Report_EO1_Worksheet = ['AF2:AF200', 'AG2:AG200']

Report_EO1_Worksheet.batch_clear(clear_range_HSD_Report_EO1_Worksheet)




# === STEP 5: SEARCH FOR ODOMETER VALUES ===
print("\n🔍 STEP 5: Searching for odometer values...")


# Get current date in expected format (e.g., "05,May25")
current_date_str = datetime.now().strftime('%d,%b%y')

# Get main data from Report_EO1
main_data = Report_EO1_Worksheet.get_all_values()
header, rows = main_data[0], main_data[1:]

# Map each row to bus + date (using today's date)
row_map = {}
bus_set = set()
for i, row in enumerate(rows, start=2):
    try:
        if len(row) > 1:  # Make sure row has at least 2 columns
            bus = re.sub(r'[^a-zA-Z0-9]', '', str(row[1]).strip()).upper()  # Clean bus name
            if bus:
                row_map[i] = (bus, current_date_str)
                bus_set.add(bus)
    except Exception as e:
        print(f"⚠️ Error processing row {i}: {e}")
        continue

print(f"🔎 Total buses to process: {len(bus_set)}")
if bus_set:
    print("Sample buses:", list(bus_set)[:5])  # Log first 5 buses

# Initialize lookup maps with thread safety
bus_date_value_map = {}
match_type_map = {}
fallback_date_map = {}
source_spreadsheet_map = {}
map_lock = threading.Lock()

# Function to find closest earlier date value
def find_closest_date_value(sheet_data, target_date_str, value_column_index=6):
    """Find closest earlier date with a value in the specified column"""
    if not sheet_data:
        return None, ""

    try:
        target_date = datetime.strptime(target_date_str, '%d,%b%y')
        valid_rows = []

        for row in sheet_data[1:]:  # Skip header
            if len(row) <= value_column_index:
                continue

            date_cell = row[1].strip() if len(row) > 1 else ""
            value_cell = row[value_column_index].strip() if len(row) > value_column_index else ""

            if not date_cell or not value_cell:
                continue

            # Try multiple date formats
            parsed_date = None
            for fmt in ['%d,%b%y', '%d-%b-%y', '%d/%b/%y']:
                try:
                    parsed_date = datetime.strptime(date_cell, fmt)
                    break
                except:
                    continue

            if parsed_date and parsed_date <= target_date and value_cell:
                valid_rows.append((parsed_date, date_cell, value_cell))

        if valid_rows:
            valid_rows.sort(key=lambda x: x[0], reverse=True)  # Newest date first
            best = valid_rows[0]
            return best[2], best[1]  # value, fallback_date

    except Exception as e:
        print(f"⚠️ Error in fallback for {target_date_str}: {e}")

    return None, ""

# Process a single bus to find odometer value
def process_bus_odometer(bus):
    """Process odometer data for a single bus"""
    found = False

    # Use the cached function to get worksheet data
    data, source = get_worksheet_data("All", bus)

    if data:
        found = True
        # Check for exact date match
        exact_match = False
        for row in data[1:]:  # Skip header
            if len(row) > 12 and row[1].strip() == current_date_str and row[12].strip():
                odometer_val = row[12].strip()
                if odometer_val and odometer_val.isdigit():  # Validate numeric
                    with map_lock:
                        bus_date_value_map[(bus, current_date_str)] = odometer_val
                        match_type_map[(bus, current_date_str)] = "exact"
                        fallback_date_map[(bus, current_date_str)] = current_date_str  # Store current date
                        source_spreadsheet_map[(bus, current_date_str)] = source
                    exact_match = True
                    print(f"📌 Bus {bus}: Exact match: {odometer_val} km on {current_date_str}")
                    break

        if not exact_match:
            # Fallback to closest date
            val, fallback_date = find_closest_date_value(data, current_date_str, 6)
            if val:
                with map_lock:
                    bus_date_value_map[(bus, current_date_str)] = val
                    match_type_map[(bus, current_date_str)] = "fallback"
                    fallback_date_map[(bus, current_date_str)] = fallback_date
                    source_spreadsheet_map[(bus, current_date_str)] = source
                print(f"🔄 Bus {bus}: Fallback match: {val} km on {fallback_date}")

    if not found:
        print(f"🚨 Bus {bus} not found in ANY spreadsheet!")

    return found

# Process buses in parallel batches
MAX_WORKERS = 5  # Adjust based on API limits
BATCH_SIZE = 10  # Process this many buses at once

def process_buses_in_batches(buses):
    """Process buses in parallel batches"""
    total = len(buses)
    completed = 0

    bus_chunks = chunk_list(list(buses), BATCH_SIZE)

    for chunk in bus_chunks:
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(chunk))) as executor:
            futures = {executor.submit(process_bus_odometer, bus): bus for bus in chunk}
            for future in concurrent.futures.as_completed(futures):
                bus = futures[future]
                try:
                    result = future.result()
                    completed += 1
                    if completed % 10 == 0 or completed == total:
                        print(f"Progress: {completed}/{total} buses processed ({completed/total:.1%})")
                except Exception as e:
                    print(f"⚠️ Error processing bus {bus}: {e}")

        # Small delay between batches to avoid API rate limits
        time.sleep(1)

    print(f"✅ Completed processing all {total} buses")

# Process all buses in batches
if bus_set:
    process_buses_in_batches(bus_set)

# Update Report_EO1 with logging
print("\n📝 Writing data to Report_EO1...")
updated_rows = []
for row_idx, (bus, date_str) in row_map.items():
    key = (bus, date_str)
    val = bus_date_value_map.get(key, "")
    fallback = fallback_date_map.get(key, "")
    match_type = match_type_map.get(key, "")
    source = source_spreadsheet_map.get(key, "")

    # Improved logic for Column C (date) and Column D (value)
    if match_type == "exact":
        # For exact matches, use the current date (date_str)
        col_c = date_str
        col_d = val if val else ""
    elif match_type == "fallback":
        # For fallback matches, use the fallback date we found
        col_c = fallback if fallback else ""
        col_d = val if val else ""
    else:
        # No match found
        col_c = ""
        col_d = ""

    # Add to batch update
    updated_rows.append([col_c, col_d])

# Prepare batch update
if updated_rows:
    # Split into manageable chunks
    MAX_ROWS_PER_UPDATE = 1000  # Adjust based on API limits
    row_chunks = chunk_list(updated_rows, MAX_ROWS_PER_UPDATE)

    for i, chunk in enumerate(row_chunks):
        start_row = 2 + i * MAX_ROWS_PER_UPDATE
        end_row = start_row + len(chunk) - 1
        range_to_update = f"AF{start_row}:AG{end_row}"

        Report_EO1_Worksheet.update(range_to_update, chunk)
        print(f"✅ Updated rows {start_row}-{end_row} in Report_EO1")

        # Small delay between batch updates
        if i < len(row_chunks) - 1:
            time.sleep(1)

    print(f"✅ All {len(updated_rows)} rows updated successfully in Report_EO1!")
else:
    print("❌ No updates made to Report_EO1.")

print("🎉 Complete script execution finished successfully!")




🚀 Initializing performance optimizations...

🧹 STEP 1: Clearing data...

🔍 STEP 5: Searching for odometer values...
🔎 Total buses to process: 84
Sample buses: ['461', '521', '613', '441', '507']
📑 Found worksheet 461 in Odometer_FG
📑 Found worksheet 521 in Odometer_SVP
🔄 Bus 521: Fallback match: 75 km on 13,May25
📑 Found worksheet 613 in Odometer_SVP
🔄 Bus 613: Fallback match: 160 km on 17,Mar25
📑 Found worksheet 441 in Odometer_SVP
🔄 Bus 441: Fallback match: 21 km on 03,Mar25
📑 Found worksheet 507 in Odometer_FG
📑 Found worksheet 585 in Odometer_SVP
🔄 Bus 585: Fallback match: 84 km on 14,May25
📑 Found worksheet 538 in Odometer_SVP
🔄 Bus 538: Fallback match: 71 km on 13,May25
📑 Found worksheet 474 in Odometer_SVP
🔄 Bus 474: Fallback match: 75 km on 15,May25
📑 Found worksheet 463 in Odometer_FG
📑 Found worksheet 592 in Odometer_FG
Progress: 10/84 buses processed (11.9%)
📑 Found worksheet 587 in Odometer_SVP
🔄 Bus 587: Fallback match: 132 km on 14,May25
📑 Found worksheet 486 in Odometer_

<ipython-input-9-e68580c3998b>:295: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  Report_EO1_Worksheet.update(range_to_update, chunk)


✅ Updated rows 2-85 in Report_EO1
✅ All 84 rows updated successfully in Report_EO1!
🎉 Complete script execution finished successfully!
